<a href="https://colab.research.google.com/github/Rafael-Maranhao/portfolio/blob/main/estudo_de_correla%C3%A7%C3%A3o_salarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## 📊 Importação de Bibliotecas

In [ ]:
import pandas as pd

## 🗃️ Carregamento dos dados

In [ ]:
df = pd.read_csv('dataset_salarios.csv')

print("Colunas disponíveis no dataset:")
print(df.columns.tolist())

Colunas disponíveis no dataset:
['work_year', 'experience_level', 'employment_type', 'job_title', 'salary', 'salary_currency', 'salary_in_usd', 'employee_residence', 'remote_ratio', 'company_location', 'company_size']


## 🧹 Pré-processamento dos dados

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 1. Codifica título do cargo
encoder_job = LabelEncoder()
df['cargo_encoded'] = encoder_job.fit_transform(df['job_title'])

# 2. Codifica tamanho da empresa
encoder_size = LabelEncoder()
df['tamanho_empresa_encoded'] = encoder_size.fit_transform(df['company_size'])

# 3. Converte remote_ratio para binário (100 = remoto, outros = não-remoto)
df['remoto_binario'] = df['remote_ratio'].apply(lambda x: 1 if x == 100 else 0)

## 🔍 Seleção de Variáveis

In [ ]:
# Define variáveis independentes (X) e dependente (y)
X = df[['cargo_encoded', 'tamanho_empresa_encoded', 'remoto_binario']]
y = df['salary_in_usd']

## 📈 Modelagem e *Cálculos*



In [ ]:
from sklearn.linear_model import LinearRegression
modelo = LinearRegression()
modelo.fit(X, y)
r_quadrado = modelo.score(X, y)

def calcula_r2_variavel(X_col, y):
    modelo = LinearRegression()
    modelo.fit(X_col.values.reshape(-1, 1), y)
    return modelo.score(X_col.values.reshape(-1, 1), y)

r2_job = calcula_r2_variavel(df['cargo_encoded'], y)
r2_size = calcula_r2_variavel(df['tamanho_empresa_encoded'], y)
r2_remoto = calcula_r2_variavel(df['remoto_binario'], y)

# Regressão multivariada
modelo = LinearRegression()
modelo.fit(X, y)

# Coeficientes
coeficientes = pd.Series(modelo.coef_, index=X.columns)

from scipy import stats
import numpy as np

# Calcula p-valores manualmente
X_with_const = np.column_stack([np.ones(len(X)), X])  # Adiciona intercepto
params = np.append(modelo.intercept_, modelo.coef_)
predictions = modelo.predict(X)

mse = np.sum((y - predictions) ** 2) / (X_with_const.shape[0] - X_with_const.shape[1])
var_b = mse * np.linalg.inv(np.dot(X_with_const.T, X_with_const)).diagonal()
sd_b = np.sqrt(var_b)
t_b = params / sd_b
p_values = [2 * (1 - stats.t.cdf(np.abs(i), (len(X) - len(X_with_const[0])))) for i in t_b]

# Cria uma tabela resumo
import pandas as pd
coef_df = pd.DataFrame({
    "Variável": ["Intercepto"] + list(X.columns),
    "Coeficiente": params,
    "Erro Padrão": sd_b,
    "t-stat": t_b,
    "p-valor": p_values
})

## 🎯 Resultados

In [ ]:
print(f"\nR² do modelo: {r_quadrado:.4f} ({r_quadrado*100:.2f}%)\n")  # Corrigido formatação

print("\nVariáveis consideradas:\n")  # Adicionado \n antes e depois

print(f"R² - Título do cargo: {r2_job:.4f}\n")  # \n no final
print(f"R² - Tamanho da empresa: {r2_size:.4f}\n")
print(f"R² - Trabalho remoto: {r2_remoto:.4f}\n")

print("\n★ Coeficientes da regressão multivariada:\n")  # Espaçamento extra
print(coeficientes.to_string())  # Melhor formatação para Series
print("\n" + coef_df.to_string())  # DataFrame com quebra de linha


R² do modelo: 0.0523 (5.23%)


Variáveis consideradas:

R² - Título do cargo: 0.0507

R² - Tamanho da empresa: 0.0001

R² - Trabalho remoto: 0.0017


★ Coeficientes da regressão multivariada:

cargo_encoded               438.966428
tamanho_empresa_encoded   -7633.750849
remoto_binario            -3587.501049

                  Variável    Coeficiente  Erro Padrão     t-stat   p-valor
0               Intercepto  125730.752122  2141.027518  58.724491  0.000000
1            cargo_encoded     438.966428    14.791164  29.677612  0.000000
2  tamanho_empresa_encoded   -7633.750849  1950.066772  -3.914610  0.000091
3           remoto_binario   -3587.501049  1123.275393  -3.193786  0.001407


## 💾 Exportação dos Resultados

In [ ]:
df.to_csv('salarios_processados.csv', index=False)
print("\nDataset processado salvo como 'salarios_processados.csv'")


Dataset processado salvo como 'salarios_processados.csv'
